<a href="https://colab.research.google.com/github/nonyeezeh/Research-Project-Code/blob/main/LBN_Sparse_BIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.sampling import BayesianModelSampling
from sklearn.preprocessing import LabelEncoder
from pgmpy.estimators import HillClimbSearch, BicScore, MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
from sklearn.metrics import accuracy_score
from scipy.stats import entropy
import os
from tabulate import tabulate
from sklearn.model_selection import train_test_split
from pgmpy.estimators import AICScore

# Bayesian Network Data Generation 1000, 2000, ..., 10000 Samples (sparse)

In [3]:
# Function to generate CPDs for a sparse BN
def generate_sparse_cpds():
    # Generate random probabilities for IR
    ir_probs = np.random.rand(3)
    ir_probs /= ir_probs.sum()  # Normalize to make it a valid probability distribution

    # Generate random probabilities for EI given IR
    ei_given_ir_probs = np.random.rand(3, 3)
    ei_given_ir_probs /= ei_given_ir_probs.sum(axis=0, keepdims=True)

    # Randomly decide the relationship for SP to simulate sparsity
    relationships = ['IR', 'EI', 'IR_EI']
    relationship = np.random.choice(relationships)

    if relationship == 'IR_EI':
        sp_probs = np.random.rand(3, 3, 3)
        sp_probs /= sp_probs.sum(axis=0, keepdims=True)
    elif relationship == 'IR':
        sp_probs = np.random.rand(3, 3)
        sp_probs /= sp_probs.sum(axis=0, keepdims=True)
    elif relationship == 'EI':
        sp_probs = np.random.rand(3, 3)
        sp_probs /= sp_probs.sum(axis=0, keepdims=True)

    return ir_probs, ei_given_ir_probs, sp_probs, relationship

# Function to generate and save samples for a sparse BN
def generate_and_save_sparse_samples(ir_probs, ei_probs, sp_probs, relationship, sample_size, filename):
    output_data = []

    # Generate `sample_size` random samples
    for _ in range(sample_size):
        # Sample `IR` state based on `IR` probabilities
        ir_state_idx = np.random.choice(3, p=ir_probs)
        ir_state = ['low', 'medium', 'high'][ir_state_idx]
        ir_prob = ir_probs[ir_state_idx]

        # Sample `EI` state based on `EI` probabilities given `IR`
        ei_probs_given_ir = ei_probs[:, ir_state_idx]
        ei_state_idx = np.random.choice(3, p=ei_probs_given_ir)
        ei_state = ['poor', 'average', 'good'][ei_state_idx]
        ei_prob = ei_probs_given_ir[ei_state_idx]

        # Sample `SP` state based on the relationship type
        if relationship == 'IR_EI':
            sp_probs_given_ir_ei = sp_probs[:, ir_state_idx, ei_state_idx]
        elif relationship == 'IR':
            sp_probs_given_ir_ei = sp_probs[:, ir_state_idx]
        elif relationship == 'EI':
            sp_probs_given_ir_ei = sp_probs[:, ei_state_idx]

        sp_state_idx = np.random.choice(3, p=sp_probs_given_ir_ei)
        sp_state = ['decrease', 'stable', 'increase'][sp_state_idx]
        sp_prob = sp_probs_given_ir_ei[sp_state_idx]

        # Append sample data to output list
        output_data.append({
            'IR_State': ir_state,
            'IR_Prob': f'{ir_prob:.4f}',
            'EI_State': ei_state,
            'EI_Prob': f'{ei_prob:.4f}',
            'SP_Probabilities (decrease, stable, increase)': ', '.join([f'{prob:.4f}' for prob in sp_probs_given_ir_ei]),
            'Chosen_SP_State': sp_state,
            'Chosen_SP_Probability': f'{sp_prob:.4f}',
            'Relationship': relationship
        })

    # Create a DataFrame from the output data
    output_df = pd.DataFrame(output_data)

    # Save the output DataFrame to a CSV file
    output_df.to_csv(filename, index=False)

    # Print the first few rows for visual confirmation
    print(f"\nSample size: {sample_size} - First few rows of generated samples:\n")
    print(tabulate(output_df.head(), headers='keys', tablefmt='grid'))

# Generate and save samples for sample sizes from 1000 to 10000 every 1000
sample_sizes = range(1000, 11000, 1000)

for size in sample_sizes:
    # Generate the CPDs for Sparse BN
    ir_probs, ei_given_ir_probs, sp_probs, relationship = generate_sparse_cpds()

    # Generate and save individual samples for the given sample size
    generate_and_save_sparse_samples(ir_probs, ei_given_ir_probs, sp_probs, relationship, size, f'combined_sparse_probabilities_{size}.csv')

# Notify the user that the process is done
print("\nGeneration and saving of individual samples complete for all sample sizes (Sparse BN)!")


Sample size: 1000 - First few rows of generated samples:

+----+------------+-----------+------------+-----------+-------------------------------------------------+-------------------+-------------------------+----------------+
|    | IR_State   |   IR_Prob | EI_State   |   EI_Prob | SP_Probabilities (decrease, stable, increase)   | Chosen_SP_State   |   Chosen_SP_Probability | Relationship   |
+====+============+===========+============+===========+=================================================+===================+=========================+================+
|  0 | medium     |    0.5243 | average    |    0.5248 | 0.2872, 0.4177, 0.2951                          | stable            |                  0.4177 | IR             |
+----+------------+-----------+------------+-----------+-------------------------------------------------+-------------------+-------------------------+----------------+
|  1 | medium     |    0.5243 | average    |    0.5248 | 0.2872, 0.4177, 0.2951            

# LBN Sparse BIC

In [4]:
# Sample sizes to loop through
sample_sizes = range(1000, 11000, 1000)

# Loop through each sample size
for sample_size in sample_sizes:
    print(f"\nProcessing sample size: {sample_size}")

    # Load the sparse dataset for the current sample size
    sparse_data_file = f'combined_sparse_probabilities_{sample_size}.csv'
    df_sparse = pd.read_csv(sparse_data_file)

    # Manually encode categorical variables for IR, EI, and SP
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}
    sp_map = {'decrease': 0, 'stable': 1, 'increase': 2}

    df_sparse['IR_encoded'] = df_sparse['IR_State'].map(ir_map)
    df_sparse['EI_encoded'] = df_sparse['EI_State'].map(ei_map)
    df_sparse['SP_encoded'] = df_sparse['Chosen_SP_State'].map(sp_map)

    # Split the data into training (70%) and test (30%)
    X = df_sparse[['IR_encoded', 'EI_encoded']]
    y = df_sparse['SP_encoded']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

    # Concatenate X and y to form the training set for learning the BN structure
    df_train = pd.concat([X_train, y_train], axis=1)

    # Define the Hill-Climb structure learning algorithm
    hc = HillClimbSearch(df_train)
    scoring_method = BicScore(df_train)

    # Estimate the best structure
    best_dag = hc.estimate(scoring_method=scoring_method)
    best_model = BayesianNetwork(best_dag.edges())

    # Display the learned structure (edges of the Bayesian Network)
    print(f"\nLearned Structure (Edges) for {sample_size} samples:")
    print(best_model.edges())

    # Learn the CPDs using Maximum Likelihood Estimation (MLE)
    best_model.fit(df_train, estimator=MaximumLikelihoodEstimator)

    # Check if the model is valid after learning the parameters
    assert best_model.check_model()

    # Print the learned CPDs (Conditional Probability Distributions)
    for cpd in best_model.get_cpds():
        print("\nCPD of", cpd.variable)
        print(cpd)

# Create an inference object for the best model (for K-L divergence calculation later)
inference = VariableElimination(best_model)

print("\nProcessing complete for all sample sizes (Sparse BN).")


Processing sample size: 1000


  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 1000 samples:
[('IR_encoded', 'EI_encoded'), ('SP_encoded', 'IR_encoded')]

CPD of IR_encoded
+---------------+---------------------+----------------------+---------------------+
| SP_encoded    | SP_encoded(0)       | SP_encoded(1)        | SP_encoded(2)       |
+---------------+---------------------+----------------------+---------------------+
| IR_encoded(0) | 0.3053691275167785  | 0.3224489795918367   | 0.17197452229299362 |
+---------------+---------------------+----------------------+---------------------+
| IR_encoded(1) | 0.40939597315436244 | 0.6326530612244898   | 0.6496815286624203  |
+---------------+---------------------+----------------------+---------------------+
| IR_encoded(2) | 0.28523489932885904 | 0.044897959183673466 | 0.17834394904458598 |
+---------------+---------------------+----------------------+---------------------+

CPD of EI_encoded
+---------------+---------------------+----------------------+----------------------+
| IR_

  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 2000 samples:
[('IR_encoded', 'SP_encoded'), ('IR_encoded', 'EI_encoded')]

CPD of IR_encoded
+---------------+-----------+
| IR_encoded(0) | 0.0757143 |
+---------------+-----------+
| IR_encoded(1) | 0.402143  |
+---------------+-----------+
| IR_encoded(2) | 0.522143  |
+---------------+-----------+

CPD of SP_encoded
+---------------+---------------------+---------------------+---------------------+
| IR_encoded    | IR_encoded(0)       | IR_encoded(1)       | IR_encoded(2)       |
+---------------+---------------------+---------------------+---------------------+
| SP_encoded(0) | 0.32075471698113206 | 0.23268206039076378 | 0.3844049247606019  |
+---------------+---------------------+---------------------+---------------------+
| SP_encoded(1) | 0.33962264150943394 | 0.4849023090586146  | 0.15868673050615595 |
+---------------+---------------------+---------------------+---------------------+
| SP_encoded(2) | 0.33962264150943394 | 0.2824156305506217

...


Learned Structure (Edges) for 8000 samples:
[('IR_encoded', 'EI_encoded'), ('SP_encoded', 'EI_encoded'), ('SP_encoded', 'IR_encoded')]

CPD of IR_encoded
+---------------+---------------------+---------------------+--------------------+
| SP_encoded    | SP_encoded(0)       | SP_encoded(1)       | SP_encoded(2)      |
+---------------+---------------------+---------------------+--------------------+
| IR_encoded(0) | 0.3081732916480572  | 0.33980582524271846 | 0.5714285714285714 |
+---------------+---------------------+---------------------+--------------------+
| IR_encoded(1) | 0.18133095131755247 | 0.22694174757281554 | 0.0922384701912261 |
+---------------+---------------------+---------------------+--------------------+
| IR_encoded(2) | 0.5104957570343903  | 0.433252427184466   | 0.3363329583802025 |
+---------------+---------------------+---------------------+--------------------+

CPD of EI_encoded
+---------------+---------------------+-----+----------------------+-----------

  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 9000 samples:
[('IR_encoded', 'EI_encoded'), ('SP_encoded', 'IR_encoded')]

CPD of IR_encoded
+---------------+----------------------+---------------------+----------------------+
| SP_encoded    | SP_encoded(0)        | SP_encoded(1)       | SP_encoded(2)        |
+---------------+----------------------+---------------------+----------------------+
| IR_encoded(0) | 0.48024871982443307  | 0.23895169578622816 | 0.006790123456790123 |
+---------------+----------------------+---------------------+----------------------+
| IR_encoded(1) | 0.5036576444769568   | 0.6546762589928058  | 0.7197530864197531   |
+---------------+----------------------+---------------------+----------------------+
| IR_encoded(2) | 0.016093635698610095 | 0.10637204522096609 | 0.2734567901234568   |
+---------------+----------------------+---------------------+----------------------+

CPD of EI_encoded
+---------------+---------------------+---------------------+---------------------

  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 10000 samples:
[('IR_encoded', 'SP_encoded'), ('EI_encoded', 'IR_encoded')]

CPD of IR_encoded
+---------------+---------------------+---------------------+---------------------+
| EI_encoded    | EI_encoded(0)       | EI_encoded(1)       | EI_encoded(2)       |
+---------------+---------------------+---------------------+---------------------+
| IR_encoded(0) | 0.07911576497963932 | 0.1464190981432361  | 0.06007067137809187 |
+---------------+---------------------+---------------------+---------------------+
| IR_encoded(1) | 0.08551483420593368 | 0.16710875331564987 | 0.05742049469964664 |
+---------------+---------------------+---------------------+---------------------+
| IR_encoded(2) | 0.835369400814427   | 0.686472148541114   | 0.8825088339222615  |
+---------------+---------------------+---------------------+---------------------+

CPD of SP_encoded
+---------------+---------------------+--------------------+---------------------+
| IR_encoded    

# Entropy

In [7]:
# Sample sizes to loop through
sample_sizes = range(1000, 11000, 1000)

# Prepare a list to store K-L divergence results
kl_divergence_results = []

# Loop through each sample size
for sample_size in sample_sizes:
    print(f"\nProcessing K-L Divergence for sample size: {sample_size}")

    # Load the sparse dataset used in the LBN part
    sparse_data_file = f'combined_sparse_probabilities_{sample_size}.csv'
    df_sparse = pd.read_csv(sparse_data_file)

    # Manually encode categorical variables for IR, EI, and SP
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}
    sp_map = {'decrease': 0, 'stable': 1, 'increase': 2}

    df_sparse['IR_encoded'] = df_sparse['IR_State'].map(ir_map)
    df_sparse['EI_encoded'] = df_sparse['EI_State'].map(ei_map)
    df_sparse['SP_encoded'] = df_sparse['Chosen_SP_State'].map(sp_map)

    # Use the test data split obtained from the LBN part
    X_test = df_sparse[['IR_encoded', 'EI_encoded']].iloc[y_test.index]
    y_test = df_sparse['SP_encoded'].iloc[y_test.index]

    # Placeholder to store K-L divergence values
    kl_divergences = []

    # Loop through each row in the test data to make predictions
    for index, row in X_test.iterrows():
        sample_input = {'IR_encoded': int(row['IR_encoded']), 'EI_encoded': int(row['EI_encoded'])}

        # Perform inference using the learned Bayesian model
        predicted_sp_distribution = inference.query(variables=['SP_encoded'], evidence=sample_input)
        predicted_probs = predicted_sp_distribution.values

        # Extract the ground truth probabilities for SP from `y_test`
        ground_truth_probabilities_str = df_sparse['SP_Probabilities (decrease, stable, increase)'].iloc[index]
        ground_truth_probs = np.array(list(map(float, ground_truth_probabilities_str.strip('[]').split(','))))

        # Ensure the probabilities are non-zero to avoid division by zero
        epsilon = 1e-10
        ground_truth_probs = np.clip(ground_truth_probs, epsilon, 1)
        predicted_probs = np.clip(predicted_probs, epsilon, 1)

        # Normalize both probability distributions
        ground_truth_probs /= ground_truth_probs.sum()
        predicted_probs /= predicted_probs.sum()

        # Calculate the K-L divergence (Learned BN vs Ground Truth)
        kl_div = entropy(ground_truth_probs, predicted_probs)
        kl_divergences.append(kl_div)

    # Calculate the average K-L divergence and standard deviation over all test samples
    average_kl_divergence = np.mean(kl_divergences)
    std_kl_divergence = np.std(kl_divergences)

    # Append the results to the list for saving later
    kl_divergence_results.append({
        'Sample_Size': sample_size,
        'Average_KL_Divergence': average_kl_divergence,
        'Std_Dev': std_kl_divergence
    })

    # Print confirmation and result for this sample size
    print(f"\nAverage K-L Divergence for {sample_size} samples: {average_kl_divergence:.4f}, Std Dev: {std_kl_divergence:.4f}")

# Save the K-L divergence results to a CSV file
kl_divergence_df = pd.DataFrame(kl_divergence_results)
kl_divergence_df.to_csv('kl_div_LBN_sparse_bic.csv', index=False)

print("\nK-L divergence calculations complete and results saved to 'kl_div_LBN_sparse_bic.csv'.")


Processing K-L Divergence for sample size: 1000


IndexError: positional indexers are out-of-bounds